In [28]:
import sympy as sp

In [29]:
x0, x1, x2 = sp.symbols('x0 x1 x2')
x_sym = [x0, x1, x2]
x_ = [1,-1,0]

f0 = [0.5*x0**2 + x1**2 + 3*x2**2 - 2*x1*x2 + 4, 'max']

# Определяем символьные переменные

# Задаём ограничения (в символьном виде)
restricts_sym = [
    1.5*x0**2 + 4*x1**2 + 2.5*x2**2 + 2*x0*x2,
    2*x0**2 + 3*x1**2 + 0.5*x2**2 + 2*x0*x1,
    2*x0 + x1 - 2*x2
]

restrict_val = [7,3,2]
z = [[0,1,1],[-1,0,-1],[-1,1,1]]

In [30]:
def compute_derivates(variables, restrictions, selected_indices=None):
#Вычисляет градиенты для выбранных ограничений по заданным переменным.

    if selected_indices is None:
        selected_indices = range(len(restrictions))
    
    return [
        [sp.diff(restrictions[i], var) for var in variables]
        for i in selected_indices
    ]

def compute_derivate_in_func(function, variables):
#Вычисляет градиент (вектор частных производных) для заданной функции.

    return [sp.diff(function, var) for var in variables]

In [31]:
equals = []
for i, restrict in enumerate(restricts_sym):
    val = restrict.subs({x0: x_[0], x1: x_[1], x2: x_[2]})
    if val == restrict_val[i]:
        equals.append(i)

In [32]:
if not equals:
    print("Нет активных ограничений. Все направления возможны.")

In [33]:
derivates = compute_derivates([x0, x1, x2], restrictions = restricts_sym, 
                              selected_indices = equals)

In [34]:
def substitute_derivates(derivates):
#Подставляет значения x_ в градиенты.

    if isinstance(derivates[0], list):  # Если это список градиентов
        derivates_substituted = []
        for der in derivates:
            substituted_gradient = [
                expr.subs({x0: x_[0], x1: x_[1], x2: x_[2]}).evalf()
                for expr in der
            ]
            derivates_substituted.append(substituted_gradient)
        return derivates_substituted
    else:  # Если это один градиент
        return [
            expr.subs({x0: x_[0], x1: x_[1], x2: x_[2]}).evalf()
            for expr in derivates
        ]

In [35]:
derivates_substituted = substitute_derivates(derivates)

In [36]:
z0, z1, z2 = sp.symbols('z0 z1 z2')
z_sym = [z0, z1, z2]

In [37]:
def scalar_product(arr_substituted):

    z_expressions = []

    for derivate in arr_substituted:
        # Вычисляем скалярное произведение derivate и z_sym
        product = sum(d * z for d, z in zip(derivate, z_sym))
        z_expressions.append(product)

    return z_expressions

In [38]:
def get_results(arr_substituted):
    z_expressions = scalar_product(arr_substituted)
    results = []

    for expr in z_expressions:
        expr_results = []  # Результаты для текущего выражения
        
        for z_vec in z:
            # Подставляем z0, z1, z2 из z в выражение
            res_substitution = expr.subs({
                z0: z_vec[0],
                z1: z_vec[1],
                z2: z_vec[2]
            })
            expr_results.append(res_substitution)
        
        results.append(expr_results) 
    
    return results

def get_func_result(func_derivate, z_vectors):
    product = sum(d * z for d, z in zip(func_derivate, z_sym))
    return [product.subs({z0: z_vec[0], z1: z_vec[1], z2: z_vec[2]}).evalf() for z_vec in z_vectors]

In [39]:
results = get_results(derivates_substituted)

In [40]:
def check_result(results, is_function=False):
    for i, res_substitution in enumerate(results):
        if is_function:
            print(f'Для целевой функции:')
            
        else:
            print(f'Для активного ограничения {equals[i] + 1}:')

        for j, direction in enumerate(res_substitution):
            direction = direction.evalf()
            if is_function and direction < 0:
                print(f'\tНаправление z{j + 1} является спусковым')
            elif not is_function and direction < 0:  # Для ограничений
                print(f'\tНаправление z{j + 1} возможно')

In [41]:
check_result(results)

Для активного ограничения 2:
	Направление z1 возможно
	Направление z2 возможно
	Направление z3 возможно


In [42]:
func_derivate = compute_derivate_in_func(f0[0], x_sym)
func_substitude_derivate = substitute_derivates(func_derivate)
func_result = get_func_result(func_substitude_derivate, z)
check_result([func_result], is_function=True)

Для целевой функции:
	Направление z2 является спусковым
	Направление z3 является спусковым
